# **Install required packages**

In [ ]:
!pip install -q tensorflow tensorflow-hub opencv-python mediapipe scipy

import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import mediapipe as mp
from scipy.spatial.transform import Rotation as R
from google.colab import files
import os

# **MoveNet model**

In [ ]:
movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4").signatures['serving_default']

# **MediaPipe model**

In [ ]:
mp_pose = mp.solutions.pose
pose_mp = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# **Keypoint Dictionary and Edge List**

In [ ]:
KEYPOINT_DICT = {
    'nose': 0, 'left_eye': 1, 'right_eye': 2, 'left_ear': 3, 'right_ear': 4,
    'left_shoulder': 5, 'right_shoulder': 6, 'left_elbow': 7, 'right_elbow': 8,
    'left_wrist': 9, 'right_wrist': 10, 'left_hip': 11, 'right_hip': 12,
    'left_knee': 13, 'right_knee': 14, 'left_ankle': 15, 'right_ankle': 16
}
EDGE_LIST = [
    (0, 1), (0, 2), (1, 3), (2, 4),   # Head
    (5, 6),                           # Shoulders
    (5, 7), (7, 9),                   # Left arm
    (6, 8), (8, 10),                  # Right arm
    (11, 12),                         # Hips
    (11, 13), (13, 15),               # Left leg
    (12, 14), (14, 16)                # Right leg
]

# **MoveNet**

In [ ]:
def correct_movenet_orientation(keypoints, frame_shape):
    h, w = frame_shape
    corrected = keypoints.copy()
    shoulder_vec = keypoints[6] - keypoints[5]
    angle = np.degrees(np.arctan2(shoulder_vec[1], shoulder_vec[0]))
    if abs(angle) < 45:
        rotation_angle = -angle
        rot = R.from_euler('z', rotation_angle, degrees=True)
        torso_center = (keypoints[5] + keypoints[6] + keypoints[11] + keypoints[12]) / 4
        centered = keypoints - torso_center
        corrected = rot.apply(centered) + torso_center
        corrected[:, 0] = np.clip(corrected[:, 0], 0, w-1)
        corrected[:, 1] = np.clip(corrected[:, 1], 0, h-1)
    if corrected[5][1] > corrected[11][1]:
        corrected[:, 1] = h - corrected[:, 1]
    return corrected

def get_movenet_keypoints(frame):
    h, w = frame.shape[:2]
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_image = tf.image.resize_with_pad(tf.expand_dims(frame_rgb, axis=0), 256, 256)
    input_image = tf.cast(input_image, dtype=tf.int32)
    outputs = movenet(input_image)
    kps = outputs['output_0'].numpy()[0, 0, :, :]
    keypoints = np.zeros((17,2), dtype=np.float32)
    keypoints[:, 0] = kps[:,1] * w  # x
    keypoints[:, 1] = kps[:,0] * h  # y
    scores = kps[:,2]
    keypoints = correct_movenet_orientation(keypoints, (h, w))
    return keypoints, scores

# **MediaPipe**

In [ ]:
def get_mediapipe_keypoints(frame):
    h, w = frame.shape[:2]
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_mp.process(frame_rgb)
    keypoints = np.zeros((17, 2), dtype=np.float32)
    scores = np.zeros(17, dtype=np.float32)
    if results.pose_landmarks:
        mapping = [
            mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.RIGHT_EYE,
            mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.RIGHT_EAR,
            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
            mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.RIGHT_ELBOW,
            mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST,
            mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
            mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE,
            mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.RIGHT_ANKLE,
        ]
        for i, idx in enumerate(mapping):
            lm = results.pose_landmarks.landmark[idx]
            keypoints[i] = [lm.x * w, lm.y * h]
            scores[i] = lm.visibility if hasattr(lm, "visibility") else 1.0
    return keypoints, scores

In [ ]:
def draw_skeleton(frame, keypoints, scores, label, point_color, edge_color):
    h, w = frame.shape[:2]
    # Draw edges
    for (i, j) in EDGE_LIST:
        if scores[i] > 0.3 and scores[j] > 0.3:
            pt1 = tuple(np.round(keypoints[i]).astype(int))
            pt2 = tuple(np.round(keypoints[j]).astype(int))
            cv2.line(frame, pt1, pt2, edge_color, 2)
    # Draw points
    for i, (x, y) in enumerate(keypoints):
        if scores[i] > 0.3:
            cv2.circle(frame, (int(x), int(y)), 5, point_color, -1)
    # Label
    cv2.rectangle(frame, (0,0), (w,40), (40,40,40), -1)
    cv2.putText(frame, label, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2)
    return frame

In [ ]:
def process_video_dual(input_path, output_path_mp, output_path_mn):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out_mp = cv2.VideoWriter(output_path_mp, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
    out_mn = cv2.VideoWriter(output_path_mn, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Processing {frame_count} frames...")
    idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # MediaPipe: Red points, white edges
        mp_kps, mp_scores = get_mediapipe_keypoints(frame)
        mp_frame = frame.copy()
        mp_frame = draw_skeleton(mp_frame, mp_kps, mp_scores, "MediaPipe Pose", (0,0,255), (255,255,255))
        out_mp.write(mp_frame)
        # MoveNet: Blue points, white edges
        mn_kps, mn_scores = get_movenet_keypoints(frame)
        mn_frame = frame.copy()
        mn_frame = draw_skeleton(mn_frame, mn_kps, mn_scores, "MoveNet Pose", (255,0,0), (255,255,255))
        out_mn.write(mn_frame)
        idx += 1
        if idx % 30 == 0: print(f"Processed {idx}/{frame_count} frames...")
    cap.release()
    out_mp.release()
    out_mn.release()
    print(f"MediaPipe output saved to: {output_path_mp}")
    print(f"MoveNet output saved to: {output_path_mn}")

In [ ]:
def main():
    print("Pose Estimation Videos: MediaPipe (red points) and MoveNet Thunder (blue points), both with white edges")
    print("Please upload your video file (preferably MP4):")
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("No video uploaded")
    video_path = list(uploaded.keys())[0]
    output_path_mp = "pose_mediapipe.mp4"
    output_path_mn = "pose_movenet.mp4"
    print("Processing video. Please wait...")
    process_video_dual(video_path, output_path_mp, output_path_mn)
    print("Download MediaPipe result:")
    files.download(output_path_mp)
    print("Download MoveNet result:")
    files.download(output_path_mn)

if __name__ == "__main__":
    main()